In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [4]:
train_data=pd.read_excel(r'../input/flight-fare-prediction/Data_Train.xlsx')
train_data.head()
pd.set_option('display.max_columns',None)

In [5]:
train_data.info()

In [6]:
train_data.columns

In [7]:
train_data['Duration'].value_counts()

In [8]:
train_data.dropna(inplace=True)

In [9]:
train_data.isnull().sum()

In [10]:
train_data.shape

In [11]:
#import dtale

In [12]:
#dtale.show(train_data)

## EDA

In [13]:
train_data['journey_day']=pd.to_datetime(train_data.Date_of_Journey, format='%d/%m/%Y').dt.day

In [14]:
train_data['journey_month']=pd.to_datetime(train_data.Date_of_Journey, format='%d/%m/%Y').dt.month

In [15]:
train_data.head()

In [16]:
type('journey_day')

In [17]:
train_data.drop(['Date_of_Journey'],axis=1,inplace=True)

In [18]:
train_data['Dep_hours']=pd.to_datetime(train_data['Dep_Time']).dt.hour
train_data['Dep_min']=pd.to_datetime(train_data['Dep_Time']).dt.minute


train_data.drop(['Dep_Time'],axis=1,inplace=True)

In [19]:
train_data.head()

In [20]:
train_data['Arrival_hour']=pd.to_datetime(train_data['Arrival_Time']).dt.hour
train_data['Arrival_min']=pd.to_datetime(train_data['Arrival_Time']).dt.minute


train_data.drop(['Arrival_Time'],axis=1,inplace=True)

In [21]:
train_data.head()

In [22]:
# Time taken by plane to reach destination is called Duration
# It is the differnce betwwen Departure Time and Arrival time


# Assigning and converting Duration column into list
duration = list(train_data["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))   # Extracts only minutes from duration

In [23]:
train_data['Duration_hours']=duration_hours
train_data['Duration_mins']=duration_mins

In [24]:
train_data.head()

In [25]:
train_data.drop(['Duration'],axis=1,inplace=True)

In [26]:
train_data.head()

#### Handling Categorical Data

One can find many ways to handle categorical data. Some of them categorical data are,

1. **Nominal data** --> data are not in any order --> **OneHotEncoder** is used in this case
2. **Ordinal data** --> data are in order --> **LabelEncoder** is used in this case

In [27]:
train_data['Airline'].value_counts()

In [28]:
# From graph we can see that Jet Airways Business have the highest Price.
# Apart from the first Airline almost all are having similar median

# Airline vs Price

In [29]:
#dtale.show(train_data)

In [30]:
# From graph we can see that Jet Airways Business have the highest Price.
# Apart from the first Airline almost all are having similar median

# airline vs price
sns.catplot(y='Price',x='Airline',data=train_data.sort_values('Price',ascending=False),kind='boxen',height=6,aspect=3)

In [31]:
# as Airline is nominal categorical data we will perform onehotencoding
Airline=train_data['Airline']
Airline=pd.get_dummies(Airline,drop_first=True)
Airline.head()

In [32]:
train_data['Source'].value_counts()

In [33]:
sns.catplot(y='Price',x='Source',data=train_data.sort_values('Price',ascending=False),kind='boxen',height=6,aspect=3)

In [34]:
Source=train_data['Source']
Source=pd.get_dummies(Source,drop_first=True)
Source.head()

In [35]:
train_data['Destination'].value_counts()

In [36]:
## Destination is nominal data we will perform onehotencoding
Destination=train_data['Destination']
Destination=pd.get_dummies(Destination,drop_first=True)
Destination.head()

In [37]:
train_data['Route']

In [38]:
train_data['Total_Stops'].value_counts()

In [39]:
## Additional info contains almost 80% of no info
## Route and total stopes are related to each other
train_data.drop(['Route','Additional_Info'],axis=1,inplace=True)

In [40]:
train_data.head()

In [41]:
## as this case of ordinal categorical type we perform labelencoder
# here value are assigned with corresponding keys
train_data.replace({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4},inplace=True)

In [42]:
train_data.head()

In [43]:
## concatenate dataframe ---> train data + airline + Source + Destination
data_train=pd.concat([train_data,Airline,Source,Destination],axis=1)

In [44]:
data_train.head()

In [45]:
data_train.drop(['Airline','Source','Destination'],axis=1,inplace=True)

In [46]:
data_train.head()

In [47]:
data_train.columns

### Test Data

In [48]:
test_data=pd.read_excel(r'../input/test-data/Test_set.xlsx')
test_data.head()

In [49]:
# Preprocessing

print("Test data Info")
print("-"*75)
print(test_data.info())

print()
print()

print("Null values :")
print("-"*75)
test_data.dropna(inplace = True)
print(test_data.isnull().sum())

# EDA

# Date_of_Journey
test_data["Journey_day"] = pd.to_datetime(test_data.Date_of_Journey, format="%d/%m/%Y").dt.day
test_data["Journey_month"] = pd.to_datetime(test_data["Date_of_Journey"], format = "%d/%m/%Y").dt.month
test_data.drop(["Date_of_Journey"], axis = 1, inplace = True)

# Dep_Time
test_data["Dep_hour"] = pd.to_datetime(test_data["Dep_Time"]).dt.hour
test_data["Dep_min"] = pd.to_datetime(test_data["Dep_Time"]).dt.minute
test_data.drop(["Dep_Time"], axis = 1, inplace = True)

# Arrival_Time
test_data["Arrival_hour"] = pd.to_datetime(test_data.Arrival_Time).dt.hour
test_data["Arrival_min"] = pd.to_datetime(test_data.Arrival_Time).dt.minute
test_data.drop(["Arrival_Time"], axis = 1, inplace = True)

# Duration
duration = list(test_data["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))   # Extracts only minutes from duration

# Adding Duration column to test set
test_data["Duration_hours"] = duration_hours
test_data["Duration_mins"] = duration_mins
test_data.drop(["Duration"], axis = 1, inplace = True)


# Categorical data

print("Airline")
print("-"*75)
print(test_data["Airline"].value_counts())
Airline = pd.get_dummies(test_data["Airline"], drop_first= True)

print()

print("Source")
print("-"*75)
print(test_data["Source"].value_counts())
Source = pd.get_dummies(test_data["Source"], drop_first= True)

print()

print("Destination")
print("-"*75)
print(test_data["Destination"].value_counts())
Destination = pd.get_dummies(test_data["Destination"], drop_first = True)

# Additional_Info contains almost 80% no_info
# Route and Total_Stops are related to each other
test_data.drop(["Route", "Additional_Info"], axis = 1, inplace = True)

# Replacing Total_Stops
test_data.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)

# Concatenate dataframe --> test_data + Airline + Source + Destination
data_test = pd.concat([test_data, Airline, Source, Destination], axis = 1)

data_test.drop(["Airline", "Source", "Destination"], axis = 1, inplace = True)

print()
print()

print("Shape of test data : ", data_test.shape)


In [50]:
data_test.head()

### features selection

One can use either of these 3 methods for feature selection:
1. heatmap
2. feature_importance
3. selectKBest

In [51]:
data_train.shape

In [52]:
data_train.columns

In [53]:
X=data_train.loc[:,['Total_Stops','journey_day', 'journey_month', 'Dep_hours',
       'Dep_min', 'Arrival_hour', 'Arrival_min', 'Duration_hours',
       'Duration_mins', 'Air India', 'GoAir', 'IndiGo', 'Jet Airways',
       'Jet Airways Business', 'Multiple carriers',
       'Multiple carriers Premium economy', 'SpiceJet', 'Trujet', 'Vistara',
       'Vistara Premium economy', 'Chennai', 'Delhi', 'Kolkata', 'Mumbai',
       'Cochin', 'Delhi', 'Hyderabad', 'Kolkata', 'New Delhi']]

In [54]:
X.head()

In [55]:
y=data_train.iloc[:,1]
y.head()

In [56]:
## Finds the corelation between independent and dependent variable

plt.figure(figsize=(18,18))
sns.heatmap(train_data.corr(),annot=True,cmap= 'inferno')

In [57]:
from sklearn.ensemble import ExtraTreesRegressor

In [58]:
selection=ExtraTreesRegressor()
selection.fit(X,y)

In [59]:
print(selection.feature_importances_)

In [60]:
## plot graph of features importance for better visulization
plt.figure(figsize=(12,8))
feat_importance=pd.Series(selection.feature_importances_,index=X.columns)
feat_importance.nlargest(20).plot(kind='barh')

##### Fitting model using Random Forest
1. Split dataset into train and test set in order to prediction w.r.t X_test
2. If needed do scaling of data
3. Scaling is not done in Random forest
4. Import model
5. Fit the data
6. Predict w.r.t X_test
7. In regression check RSME Score
8. Plot graph

In [61]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [62]:
from sklearn.ensemble import RandomForestRegressor
reg_rf=RandomForestRegressor()
reg_rf.fit(X_train,y_train)

In [63]:
y_pred=reg_rf.predict(X_test)

In [64]:
y_pred

In [65]:
y_test

In [66]:
reg_rf.score(X_train,y_train)

In [67]:
reg_rf.score(X_test,y_test)

In [68]:
sns.distplot(y_test-y_pred)
plt.show()

In [69]:
plt.scatter(y_test,y_pred,alpha=.6)
plt.xlabel('y_test')
plt.ylabel('y_pred')
plt.show()

In [70]:
from sklearn import metrics

In [71]:
print('MAE:',metrics.mean_absolute_error(y_test,y_pred))
print('MSE:',metrics.mean_squared_error(y_test,y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [72]:
metrics.r2_score(y_test,y_pred)

#### Hyperparameter Tuning
1. Choose following method for hyperparameter tuning
2. RandomizedSearchCV --> Fast
3. GridSearchCV
4. Assign hyperparameters in form of dictionery
5. Fit the model
6. Check best paramters and best score

In [73]:
from sklearn.model_selection import RandomizedSearchCV

In [74]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [75]:
# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [76]:
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = reg_rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2,
                               random_state=42, n_jobs = 1)

In [78]:
rf_random.fit(X_train,y_train) 

In [79]:
rf_random.best_params_

In [80]:
prediction = rf_random.predict(X_test)

In [81]:
prediction

In [82]:
y_test

In [ ]:
#dtale.show(r)

In [83]:
plt.figure(figsize=(8,8))
sns.distplot(y_test-prediction)

In [84]:
plt.figure(figsize=(8,8))
plt.scatter(y_test,prediction,alpha=.6)
plt.xlabel('y_test')
plt.ylabel('prediction')
plt.show()

# **Save the model to reuse it**

In [85]:
import pickle
# open a file, where you ant to store the data
file = open('flight_rf_1.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)

In [86]:
model=open('flight_rf_1.pkl','rb')
forest=pickle.load(model)

In [87]:
y_prediction = forest.predict(X_test)

In [88]:
metrics.r2_score(y_test,y_prediction)